In [3]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from ray.rllib.algorithms.impala import ImpalaConfig, Impala
from ray.tune.registry import register_env
from py_dss_interface import DSSDLL
import pandas as pd
import torch
import warnings
import logging
import os
import csv

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the seed for reproducibility
SEED = 42

# Initialize OpenDSS
dss = DSSDLL(r"C:\Program Files\OpenDSS")
dss_file = r"D:\Alaa_Selim\123Bus\IEEE123Master.dss"
dss.text(f"compile [{dss_file}]")

# defining lengths for each segment
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30
TRANSFORMER_TAPS_ACTION_LEN = 1
CAPACITOR_ACTION_LEN = 4

# defining values for each segment
PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80
PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100
BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100
TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1
CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1

class PowerSystemEnv(gym.Env):
    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file, seed=None, load_scale=3):
        super(PowerSystemEnv, self).__init__()
        self.seed(seed)
        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        self.ranked_buses = [
            '1', '7', '8', '13', '21', '23', '29', '250', '35', '40', '42', '55',
            '56', '65', '76', '78', '66', '79', '81', '83', '91', '95', '100',
            '197', '300', '110', '135', '160', '152', '610'
        ]
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated = 100
        self.load_scale = load_scale
        
        # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
            self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
        
        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]
    
        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)  # Skip the header
            self.load_profile = [float(row[0]) for row in reader]

        # Define action and observation space
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_LOW] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_LOW] * CAPACITOR_ACTION_LEN
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_HIGH] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_HIGH] * CAPACITOR_ACTION_LEN
            ),
            dtype=np.float32
        )

        self.observation_space = spaces.Box(low=0, high=100000, shape=(278,), dtype=np.float32)  # Modified shape
        self.current_step = 0
        self.control_steps = 0
        self.max_control_steps = 0
        self.hourly_violations_count = []
        self.all_bus_voltages = []

    def seed(self, seed=None):
        self.seed_value = seed
        np.random.seed(seed)
        torch.manual_seed(seed)
        if seed is not None:
            torch.cuda.manual_seed_all(seed)

    def _take_action(self, action):
        pv_kvar = action[:30].copy()
        pv_kw = action[30:60].copy()
        battery_kw = action[60:90].copy()
        transformer_tap = action[90].copy()
        capacitor_states = action[91:].copy()
        capacitor_states = np.round(capacitor_states)

        irradiance = self.irradiance_profile[self.current_step % 8760]
        for z in range(30):
            if pv_kw[z] <= irradiance * self.KWrated:
                pv_kw[z] = pv_kw[z]
            else:
                pv_kw[z] = irradiance * self.KWrated

        S_max = 100
        q_max1 = np.sqrt(S_max**2 - np.power(pv_kw, 2))
        pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)

        for i in range(30):
            bus = self.ranked_buses[i]
            scaled_pv_kw = pv_kw[i]
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar[i]} Pmpp={scaled_pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw[i]} kVAR=0")

        self.controller.text(f"edit Transformer.reg1a taps={transformer_tap}")

        for i, cap_name in enumerate(self.capacitor_names):
            self.controller.text(f"edit Capacitor.{cap_name} states={int(capacitor_states[i])}")

        self.controller.text("set controlmode=off")
        self.controller.text("solve")

    def step(self, action):
        self._take_action(action)
        
        # Retrieve power losses (assuming absolute values in kW and kVAR)
        losses = self.controller.circuit_losses()  # Expected to return [P_loss_kW, Q_loss_kVAR]

        if isinstance(losses, (list, tuple)) and len(losses) >= 2:
            p_loss_kw = losses[0]      # Real power loss in kW
            q_loss_kvar = losses[1]    # Reactive power loss in kVAR
        else:
            p_loss_kw = 0.0
            q_loss_kvar = 0.0
            logging.warning("[WARNING] circuit_losses() did not return expected values.")

        # Define base power for per unit conversion (ensure this matches your OpenDSS S_base)
        S_base = 100e3  # 100 MVA in kVA

        # Convert absolute losses to per unit
        p_loss_pu = p_loss_kw / S_base
        q_loss_pu = q_loss_kvar / S_base

        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        voltage_violations = sum(1 for v in all_bus_voltages if (v <= 0.95 or v >= 1.05) and v != 0)
        
        # Modify the reward function to include power losses
        reward = - (voltage_violations + p_loss_pu)
        
        self.hourly_violations_count.append(voltage_violations)
        self.all_bus_voltages.append(all_bus_voltages)
        observation = self.get_observation()
        termination = (self.control_steps >= self.max_control_steps) or (voltage_violations == 0)
        self.control_steps += 1
        return observation, reward, termination, False, {}

    def reset(self, *, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)
        if options is not None:
            print(f"Reset options: {options}")
        self.controller.text(f"compile [{dss_file}]")
        self.current_step = (self.current_step + 1) % 8760
        self.control_steps = 0
        load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
        ]
        load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).iloc[:, 0].tolist()
        load_dict = dict(zip(load_names, load_powers))
        load_scales = np.random.normal(loc=self.load_profile[self.current_step % 8760], scale=0.001, size=len(load_names))

        for load_name, load_scale in zip(load_names, load_scales):
            load_power = load_dict[load_name]
            result = load_power * load_scale * self.load_scale
            self.controller.text(f"edit Load.{load_name} kW={result}")

        irradiance = self.irradiance_profile[self.current_step % 8760]

        for i, bus in enumerate(self.ranked_buses):
            fixed_power_factor = 0.9
            pv_kw = 100 * irradiance
            power_factor_angle = np.arccos(fixed_power_factor)
            pv_kvar = pv_kw * np.tan(power_factor_angle)
            battery_kw = self.load_profile[self.current_step % 8760] * 100 * self.load_scale
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0")

        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        return self.get_observation(), {}

    def get_observation(self):
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        return np.array(all_bus_voltages).flatten()

irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"

def env_creator(env_config):
    return PowerSystemEnv(
        dss_path=r"C:\Program Files\OpenDSS",
        dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
        irradiance_csv_file=irradiance_csv_file,
        load_profile_file=load_profile_file,
        seed=SEED,
        load_scale=1.75  # Scale the load by 1.75 times
    )

register_env("PowerSystemEnv", env_creator)

# Training configuration
from ray.rllib.algorithms.impala import ImpalaConfig
from ray import air
from ray import tune

config = ImpalaConfig()
# Update the config object
config = config.training(
    lr=tune.grid_search([0.0001])
)
# Set the config object's env
config = config.resources(num_gpus=0)
config = config.rollouts(num_rollout_workers=3)
config = config.environment(env="PowerSystemEnv")

# Run the training
tune.Tuner(
    "IMPALA",
    run_config=air.RunConfig(stop={"episodes_total": 100000}),
    param_space=config.to_dict(),
).fit()


(pid=18752) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(Impala pid=18752) 2024-11-07 12:18:06,200	WARNING algorithm_config.py:656 -- Cannot create ImpalaConfig from given `config_dict`! Property __stdout_file__ not supported.
(RolloutWorker pid=56644) 2024-11-07 12:18:10,316	WARNING env.py:162 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=56644) 2024-11-07 12:18:10,393	WARNING deprecation.py:50 -- DeprecationWarning: `LearningRateSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=56644) 2024-11-07 12:18:10,394	WARNING deprecation.py:50 -- DeprecationWarning: `EntropyCoeffSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=56644) 2024-11-07 12:18:10,398	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConn

(RolloutWorker pid=56644) OpenDSS Started successfully! 
(RolloutWorker pid=56644) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=56644) 
(RolloutWorker pid=56644) 
(RolloutWorker pid=56644) Reset options: {}


(Impala pid=18752) 2024-11-07 12:18:10,517	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.multi_gpu_learner_thread.MultiGPULearnerThread` has been deprecated. This will raise an error in the future!
(Impala pid=18752) 2024-11-07 12:18:10,517	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.minibatch_buffer.MinibatchBuffer` has been deprecated. This will raise an error in the future!
(Impala pid=18752) 2024-11-07 12:18:10,517	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.learner_thread.LearnerThread` has been deprecated. This will raise an error in the future!
(Impala pid=18752) Install gputil for GPU system monitoring.
(Impala pid=18752) 2024-11-07 12:18:11,284	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.replay_ops.SimpleReplayBuffer` has been deprecated. This will raise an error in the future!
(pid=132984) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 40.0, 'cur_lr': 0.0001, 'total_loss': 27482.681640625, 'policy_loss': 27695.5546875, 'entropy': 138.00770568847656, 'entropy_coeff': 0.01, 'var_gnorm': 22.858535766601562, 'vf_loss': 954.3313598632812, 'vf_explained_var': -8.475780487060547e-05}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 500.0, 'num_grad_updates_lifetime': 201.0, 'diff_num_grad_updates_vs_sampler_policy': 0.6999999999999886}}, 'num_env_steps_sampled': 100950, 'num_env_steps_trained': 100500, 'num_agent_steps_sampled': 100950, 'num_agent_steps_trained': 100500, 'num_training_step_calls_since_last_synch_worker_weights': 5497, 'num_weight_broadcasts': 1711, 'num_samples_added_to_queue': 100500, 'learner_queue': {'size_count': 201, 'size_mean': 0.0, 'size_std': 0.0, 'size_quantiles': [0.0, 0.0, 0.0, 0.0, 0.0]}, 'timing_breakdown'

In [17]:
import os
import numpy as np
import pandas as pd
import gymnasium as gym
from ray.rllib.algorithms.impala import ImpalaConfig
from ray.rllib.algorithms.algorithm import Algorithm
from ray.tune.registry import register_env
import torch
import warnings
import logging
import csv

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the seed for reproducibility
SEED = 42

# Initialize OpenDSS
from py_dss_interface import DSSDLL
dss = DSSDLL(r"C:\Program Files\OpenDSS")
dss_file = r"D:\Alaa_Selim\123Bus\IEEE123Master.dss"
dss.text(f"compile [{dss_file}]")

# defining lengths for each segment
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30
TRANSFORMER_TAPS_ACTION_LEN = 1
CAPACITOR_ACTION_LEN = 4

# defining values for each segment
PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80
PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100
BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100
TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1
CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1

class PowerSystemEnv(gym.Env):
    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file, seed=None, load_scale=3):
        super(PowerSystemEnv, self).__init__()
        self.seed(seed)
        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        self.ranked_buses = [
            '1', '7', '8', '13', '21', '23', '29', '250', '35', '40', '42', '55',
            '56', '65', '76', '78', '66', '79', '81', '83', '91', '95', '100',
            '197', '300', '110', '135', '160', '152', '610'
        ]
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated = 100
        self.load_scale = load_scale
        
        # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
            self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
        
        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]
    
        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)  # Skip the header
            self.load_profile = [float(row[0]) for row in reader]

        # Define action and observation space
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_LOW] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_LOW] * CAPACITOR_ACTION_LEN
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_HIGH] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_HIGH] * CAPACITOR_ACTION_LEN
            ),
            dtype=np.float32
        )

        self.observation_space = spaces.Box(low=0, high=100000, shape=(278,), dtype=np.float32)
        self.current_step = 0
        self.control_steps = 0
        self.max_control_steps = 24  # Set max steps to 24
        self.hourly_violations_count = []
        self.all_bus_voltages = []

        # Load names and load powers
        self.load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
        ]
        load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).iloc[:, 0].tolist()
        self.load_dict = dict(zip(self.load_names, load_powers))

    def seed(self, seed=None):
        self.seed_value = seed
        np.random.seed(seed)
        torch.manual_seed(seed)
        if seed is not None:
            torch.cuda.manual_seed_all(seed)

    def update_loads_and_pv(self):
        # Update loads
        load_scales = np.random.normal(loc=self.load_profile[self.current_step % 8760], scale=0.001, size=len(self.load_names))
        for load_name, load_scale in zip(self.load_names, load_scales):
            load_power = self.load_dict[load_name]
            result = load_power * load_scale * self.load_scale
            self.controller.text(f"edit Load.{load_name} kW={result}")

        # Update irradiance
        irradiance = self.irradiance_profile[self.current_step % 8760]

        for i, bus in enumerate(self.ranked_buses):
            fixed_power_factor = 0.9
            pv_kw = 100 * irradiance
            power_factor_angle = np.arccos(fixed_power_factor)
            pv_kvar = pv_kw * np.tan(power_factor_angle)
            battery_kw = self.load_profile[self.current_step % 8760] * 100 * self.load_scale
            self.controller.text(f"edit PVSystem.PV{i+1} kVAR={pv_kvar} Pmpp={pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} kW={battery_kw}")

        self.controller.text("set controlmode=off")
        self.controller.text("solve")

    def _take_action(self, action):
        # Apply the action to the environment
        pv_kvar = action[:30].copy()
        pv_kw = action[30:60].copy()
        battery_kw = action[60:90].copy()
        transformer_tap = action[90].copy()
        capacitor_states = action[91:].copy()
        capacitor_states = np.round(capacitor_states)

        irradiance = self.irradiance_profile[self.current_step % 8760]
        for z in range(30):
            if pv_kw[z] <= irradiance * self.KWrated:
                pv_kw[z] = pv_kw[z]
            else:
                pv_kw[z] = irradiance * self.KWrated

        S_max = 100
        q_max1 = np.sqrt(S_max**2 - np.power(pv_kw, 2))
        pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)

        for i in range(30):
            bus = self.ranked_buses[i]
            scaled_pv_kw = pv_kw[i]
            self.controller.text(f"edit PVSystem.PV{i+1} kVAR={pv_kvar[i]} Pmpp={scaled_pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} kW={battery_kw[i]}")

        self.controller.text(f"edit Transformer.reg1a taps={transformer_tap}")

        for i, cap_name in enumerate(self.capacitor_names):
            self.controller.text(f"edit Capacitor.{cap_name} states={int(capacitor_states[i])}")

        self.controller.text("set controlmode=off")
        self.controller.text("solve")

    def step(self, action):
        # Advance the current step
        self.current_step = (self.current_step + 1) % 8760
        self.update_loads_and_pv()

        self._take_action(action)
        
        # Retrieve power losses
        losses = self.controller.circuit_losses()

        if isinstance(losses, (list, tuple)) and len(losses) >= 2:
            p_loss_kw = losses[0]      # Real power loss in kW
            q_loss_kvar = losses[1]    # Reactive power loss in kVAR
        else:
            p_loss_kw = 0.0
            q_loss_kvar = 0.0
            logging.warning("[WARNING] circuit_losses() did not return expected values.")

        # Define base power for per unit conversion
        S_base = 100e3  # 100 MVA in kVA

        # Convert absolute losses to per unit
        p_loss_pu = p_loss_kw / S_base
        q_loss_pu = q_loss_kvar / S_base

        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        voltage_violations = sum(1 for v in all_bus_voltages if (v <= 0.95 or v >= 1.05) and v != 0)
        
        # Modify the reward function to include power losses
        reward = - (voltage_violations + p_loss_pu)
        
        self.hourly_violations_count.append(voltage_violations)
        self.all_bus_voltages.append(all_bus_voltages)
        observation = self.get_observation()
        termination = (self.control_steps >= self.max_control_steps)
        self.control_steps += 1
        return observation, reward, termination, False, {
            "voltage_violations": voltage_violations,
            "p_loss_pu": p_loss_pu,
            "q_loss_pu": q_loss_pu,
            "all_bus_voltages": all_bus_voltages
        }

    def reset(self, *, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)
        if options is not None:
            print(f"Reset options: {options}")
        self.controller.text(f"compile [{dss_file}]")
        self.control_steps = 0
        self.max_control_steps = 24  # Reset max control steps to 24

        # Start from a specific hour (e.g., 0)
        self.current_step = (self.current_step + 1) % 8760

        # Update loads and PV systems for the initial step
        self.update_loads_and_pv()
        return self.get_observation(), {}

    def get_observation(self):
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        return np.array(all_bus_voltages).flatten()

irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"

def env_creator(env_config):
    return PowerSystemEnv(
        dss_path=r"C:\Program Files\OpenDSS",
        dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
        irradiance_csv_file=irradiance_csv_file,
        load_profile_file=load_profile_file,
        seed=SEED,
        load_scale=1.75
    )

register_env("PowerSystemEnv", env_creator)

# Testing Code Using the DRL Agent (IMPALA)

import os
import numpy as np
import pandas as pd
import gymnasium as gym
from ray.rllib.algorithms.impala import ImpalaConfig
from ray.rllib.algorithms.algorithm import Algorithm
from ray.tune.registry import register_env
import torch
import warnings
import logging
import csv

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Ensure that the environment and necessary imports are available
# from your previous code.

# Re-register the environment (if necessary)
register_env("PowerSystemEnv", env_creator)

# Specify the path to your checkpoint
checkpoint_path = r"C:\Users\Alaa\ray_results\IMPALA\IMPALA_PowerSystemEnv_3e6f4_00000_0_lr=0.0001_2024-11-07_12-18-02\checkpoint_000049"

# Initialize the algorithm configuration
config = ImpalaConfig()
config = config.environment(env="PowerSystemEnv")
config = config.rollouts(num_rollout_workers=0)  # No rollout workers for testing
config = config.framework("torch")

# Build the algorithm instance
algo = config.build()

# Restore the trained agent from the checkpoint
algo.restore(checkpoint_path)

# Create the environment instance
env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file,
    seed=SEED,
    load_scale=1.75
)

# Initialize lists to store data
steps = []
voltage_violations_history = []
p_loss_history = []
q_loss_history = []
all_bus_voltages_history = []

# Reset the environment
obs, info = env.reset()

# Run the agent for 24 steps
for step in range(1, 25):
    # Compute the action using the trained agent
    action = algo.compute_single_action(obs)
    # Take a step in the environment
    obs, reward, done, truncated, info = env.step(action)
    
    # Collect data
    steps.append(step)
    voltage_violations_history.append(info.get("voltage_violations", 0))
    p_loss_history.append(info.get("p_loss_pu", 0))
    q_loss_history.append(info.get("q_loss_pu", 0))
    all_bus_voltages_history.append(info.get("all_bus_voltages", []))
    
    if done:
        break

# Prepare data for CSV
data = {
    'Step': steps,
    'Voltage Violations': voltage_violations_history,
    'Real Power Loss (p.u.)': p_loss_history,
    'Reactive Power Loss (p.u.)': q_loss_history
}

# Add bus voltage columns
num_buses = len(all_bus_voltages_history[0])
for i in range(num_buses):
    data[f'Bus Voltage {i+1}'] = [voltages[i] for voltages in all_bus_voltages_history]

# Create a DataFrame
violations_df = pd.DataFrame(data)

# Specify the output CSV file path
output_path = r"C:\Users\Alaa\voltage_violations_and_bus_voltages_powers_LOSS_IMPALA.csv"

# Save to CSV
violations_df.to_csv(output_path, index=False)

print(f"Data saved to {output_path}")



2024-11-07 13:24:02,869	WARNING util.py:68 -- Install gputil for GPU system monitoring.
2024-11-07 13:24:02,891	INFO trainable.py:904 -- Restored on 127.0.0.1 from checkpoint: C:\Users\Alaa\ray_results\IMPALA\IMPALA_PowerSystemEnv_3e6f4_00000_0_lr=0.0001_2024-11-07_12-18-02\checkpoint_000049
2024-11-07 13:24:02,891	INFO trainable.py:913 -- Current state after restoring: {'_iteration': 49, '_timesteps_total': None, '_time_total': 561.4546101093292, '_episodes_total': 101250}


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 


Reset options: {}
OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 


Data saved to C:\Users\Alaa\voltage_violations_and_bus_voltages_powers_LOSS_IMPALA.csv


Exception in thread Thread-166:
Traceback (most recent call last):
  File "C:\Users\Alaa\.conda\envs\py310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\Alaa\.conda\envs\py310\lib\site-packages\ray\rllib\execution\learner_thread.py", line 78, in run
    self.step()
  File "C:\Users\Alaa\.conda\envs\py310\lib\site-packages\ray\rllib\execution\multi_gpu_learner_thread.py", line 149, in step
    buffer_idx, released = self.ready_tower_stacks_buffer.get()
  File "C:\Users\Alaa\.conda\envs\py310\lib\site-packages\ray\rllib\execution\minibatch_buffer.py", line 55, in get
    self.buffers[self.idx] = self.inqueue.get(timeout=self.timeout)
  File "C:\Users\Alaa\.conda\envs\py310\lib\queue.py", line 179, in get
    raise Empty
_queue.Empty
Exception in thread Thread-185:
Traceback (most recent call last):
  File "C:\Users\Alaa\.conda\envs\py310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\Alaa\.conda\envs\py310\lib\site-pa

In [15]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from py_dss_interface import DSSDLL
import pandas as pd
import torch
import warnings
import logging
import os
import csv

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the seed for reproducibility
SEED = 42

# Initialize OpenDSS
dss = DSSDLL(r"C:\Program Files\OpenDSS")
dss_file = r"D:\Alaa_Selim\123Bus\IEEE123Master.dss"
dss.text(f"compile [{dss_file}]")

# defining lengths for each segment
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30
TRANSFORMER_TAPS_ACTION_LEN = 1
CAPACITOR_ACTION_LEN = 4

# defining values for each segment
PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80
PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100
BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100
TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1
CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1

class PowerSystemEnv(gym.Env):
    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file, seed=None, load_scale=3):
        super(PowerSystemEnv, self).__init__()
        self.seed(seed)
        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        self.ranked_buses = [
            '1', '7', '8', '13', '21', '23', '29', '250', '35', '40', '42', '55',
            '56', '65', '76', '78', '66', '79', '81', '83', '91', '95', '100',
            '197', '300', '110', '135', '160', '152', '610'
        ]
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated = 100
        self.load_scale = load_scale
        
        # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
            self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
        
        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]
    
        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)  # Skip the header
            self.load_profile = [float(row[0]) for row in reader]

        # Define action and observation space
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_LOW] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_LOW] * CAPACITOR_ACTION_LEN
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_HIGH] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_HIGH] * CAPACITOR_ACTION_LEN
            ),
            dtype=np.float32
        )

        self.observation_space = spaces.Box(low=0, high=100000, shape=(278,), dtype=np.float32)
        self.current_step = 0
        self.control_steps = 0
        self.max_control_steps = 24  # Set max steps to 24
        self.hourly_violations_count = []
        self.all_bus_voltages = []

        # Load names and load powers
        self.load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
        ]
        load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).iloc[:, 0].tolist()
        self.load_dict = dict(zip(self.load_names, load_powers))

    def seed(self, seed=None):
        self.seed_value = seed
        np.random.seed(seed)
        torch.manual_seed(seed)
        if seed is not None:
            torch.cuda.manual_seed_all(seed)

    def update_loads_and_pv(self):
        # Update loads
        load_scales = np.random.normal(loc=self.load_profile[self.current_step % 8760], scale=0.001, size=len(self.load_names))
        for load_name, load_scale in zip(self.load_names, load_scales):
            load_power = self.load_dict[load_name]
            result = load_power * load_scale * self.load_scale
            self.controller.text(f"edit Load.{load_name} kW={result}")

        # Update irradiance
        irradiance = self.irradiance_profile[self.current_step % 8760]

        for i, bus in enumerate(self.ranked_buses):
            fixed_power_factor = 0.9
            pv_kw = 100 * irradiance
            power_factor_angle = np.arccos(fixed_power_factor)
            pv_kvar = pv_kw * np.tan(power_factor_angle)
            battery_kw = self.load_profile[self.current_step % 8760] * 100 * self.load_scale
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0")

        self.controller.text("set controlmode=off")
        self.controller.text("solve")

    def _take_action(self, action):
        # Apply the action to the environment
        # (In the non-control case, this action may be default or zero)
        pv_kvar = action[:30].copy()
        pv_kw = action[30:60].copy()
        battery_kw = action[60:90].copy()
        transformer_tap = action[90].copy()
        capacitor_states = action[91:].copy()
        capacitor_states = np.round(capacitor_states)

        # Note: In the non-control case, you might not apply any actions here

        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"edit PVSystem.PV{i+1} kVAR={pv_kvar[i]} Pmpp={pv_kw[i]}")
            self.controller.text(f"edit Storage.Battery{i+1} kW={battery_kw[i]}")

        self.controller.text(f"edit Transformer.reg1a taps={transformer_tap}")

        for i, cap_name in enumerate(self.capacitor_names):
            self.controller.text(f"edit Capacitor.{cap_name} states={int(capacitor_states[i])}")

        self.controller.text("set controlmode=off")
        self.controller.text("solve")

    def step(self, action):
        # Advance the current step
        self.current_step = (self.current_step + 1) % 8760
        self.update_loads_and_pv()

        self._take_action(action)
        
        # Retrieve power losses
        losses = self.controller.circuit_losses()

        if isinstance(losses, (list, tuple)) and len(losses) >= 2:
            p_loss_kw = losses[0]      # Real power loss in kW
            q_loss_kvar = losses[1]    # Reactive power loss in kVAR
        else:
            p_loss_kw = 0.0
            q_loss_kvar = 0.0
            logging.warning("[WARNING] circuit_losses() did not return expected values.")

        # Define base power for per unit conversion
        S_base = 100e3  # 100 MVA in kVA

        # Convert absolute losses to per unit
        p_loss_pu = p_loss_kw / S_base
        q_loss_pu = q_loss_kvar / S_base

        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        voltage_violations = sum(1 for v in all_bus_voltages if (v <= 0.95 or v >= 1.05) and v != 0)
        
        # Modify the reward function to include power losses
        reward = - (voltage_violations + p_loss_pu)
        
        self.hourly_violations_count.append(voltage_violations)
        self.all_bus_voltages.append(all_bus_voltages)
        observation = self.get_observation()
        termination = (self.control_steps >= self.max_control_steps)
        self.control_steps += 1
        return observation, reward, termination, False, {
            "voltage_violations": voltage_violations,
            "p_loss_pu": p_loss_pu,
            "q_loss_pu": q_loss_pu,
            "all_bus_voltages": all_bus_voltages
        }

    def reset(self, *, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)
        if options is not None:
            print(f"Reset options: {options}")
        self.controller.text(f"compile [{dss_file}]")
        self.current_step = (self.current_step + 1) % 8760
        self.control_steps = 0
        self.max_control_steps = 24  # Reset max control steps to 24

        # Update loads and PV systems for the initial step
        self.update_loads_and_pv()
        return self.get_observation(), {}

    def get_observation(self):
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        return np.array(all_bus_voltages).flatten()

irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"

def env_creator(env_config):
    return PowerSystemEnv(
        dss_path=r"C:\Program Files\OpenDSS",
        dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
        irradiance_csv_file=irradiance_csv_file,
        load_profile_file=load_profile_file,
        seed=SEED,
        load_scale=1.75
    )

register_env("PowerSystemEnv", env_creator)

# Testing Code for Non-Control Scenario

# Create the environment instance
env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file,
    seed=SEED,
    load_scale=1.75
)

# Initialize lists to store data
steps = []
voltage_violations_history = []
p_loss_history = []
q_loss_history = []
all_bus_voltages_history = []

# Reset the environment
obs, info = env.reset()

# Define non-control actions (e.g., zeros or default values)
# Here, we set actions to zero or mid-point values within the action space
default_action = np.concatenate([
    np.zeros(PV_KVAR_ACTION_LEN, dtype=np.float32),  # PV_kvar = 0
    np.zeros(PV_KW_ACTION_LEN, dtype=np.float32),    # PV_kw = 0
    np.zeros(BESS_KW_ACTION_LEN, dtype=np.float32),  # BESS_kw = 0
    np.array([1.0], dtype=np.float32),               # Transformer tap at 1.0 (nominal)
    np.zeros(CAPACITOR_ACTION_LEN, dtype=np.float32) # Capacitors off
])

# Run the environment for 24 steps with non-control actions
for step in range(1, 25):
    # Use the default action
    action = default_action
    # Take a step in the environment
    obs, reward, done, truncated, info = env.step(action)
    
    # Collect data
    steps.append(step)
    voltage_violations_history.append(info.get("voltage_violations", 0))
    p_loss_history.append(info.get("p_loss_pu", 0))
    q_loss_history.append(info.get("q_loss_pu", 0))
    all_bus_voltages_history.append(info.get("all_bus_voltages", []))
    
    if done:
        break

# Prepare data for CSV
data = {
    'Step': steps,
    'Voltage Violations': voltage_violations_history,
    'Real Power Loss (p.u.)': p_loss_history,
    'Reactive Power Loss (p.u.)': q_loss_history
}

# Add bus voltage columns
num_buses = len(all_bus_voltages_history[0])
for i in range(num_buses):
    data[f'Bus Voltage {i+1}'] = [voltages[i] for voltages in all_bus_voltages_history]

# Create a DataFrame
violations_df = pd.DataFrame(data)

# Specify the output CSV file path
output_path = r"C:\Users\Alaa\voltage_violations_and_bus_voltages_powers_L1.75_NonControl.csv"

# Save to CSV
violations_df.to_csv(output_path, index=False)

print(f"Data saved to {output_path}")



OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 


Data saved to C:\Users\Alaa\voltage_violations_and_bus_voltages_powers_L1.75_NonControl.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Data from the user for both Non-control and DRL-IMPALA for Real and Reactive Power Loss
data = {
    'Step': list(range(1, 25)),
    'Non-control Real Power Loss (p.u.)': [0.835951522, 0.797161488, 0.790025164, 0.81864728, 0.875646198, 0.922363993, 0.994424351, 1.088821382, 1.145485374, 1.168188665, 1.187597725, 1.158763556, 1.158974609, 1.171590184, 1.254136571, 1.605175383, 1.701149503, 1.625671772, 1.532475388, 1.407009164, 1.225145909, 1.022182239, 0.880297119, 0.803543619],
    'Non-control Reactive Power Loss (p.u.)': [1.671702139, 1.59362463, 1.579389374, 1.63688951, 1.751481515, 1.845197807, 1.990100527, 2.1795347, 2.293345825, 2.338803528, 2.377895333, 2.319964096, 2.320432873, 2.345663483, 2.511221845, 3.214922171, 3.407219612, 3.255968013, 3.069330467, 2.817987302, 2.45312861, 2.045857506, 1.760818368, 1.606566795],
    'DRL-IMPALA Real Power Loss (p.u.)': [0.7168389, 0.681658927, 0.671684092, 0.702231268, 0.74674435, 0.789428123, 0.866760718, 0.957726175, 1.010105092, 1.032256908, 1.055331261, 1.019515022, 1.01369281, 1.037245646, 1.10655502, 1.476948905, 1.558806098, 1.500877744, 1.396228996, 1.274251628, 1.087794443, 0.893599724, 0.752348725, 0.689603478],
    'DRL-IMPALA Reactive Power Loss (p.u.)': [1.429139406, 1.358379725, 1.339493134, 1.399835058, 1.490062501, 1.575601312, 1.73048093, 1.913198927, 2.024373771, 2.062790239, 2.10952112, 2.038166422, 2.026217626, 2.072937493, 2.21255684, 2.95617949, 3.120558475, 3.004359791, 2.794841576, 2.549337243, 2.174395382, 1.78445501, 1.501377205, 1.37458194]
}

df = pd.DataFrame(data)

# Plotting
x = np.arange(len(df['Step']))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(14, 8))

# Bars for Real Power Loss
bars1 = ax.bar(x - width/2, df['Non-control Real Power Loss (p.u.)'], width, label='Non-control Real Power Loss', color='skyblue')
bars2 = ax.bar(x + width/2, df['DRL-IMPALA Real Power Loss (p.u.)'], width, label='DRL-IMPALA Real Power Loss', color='blue')

# Adding some fancy styling
ax.set_xlabel('Step')
ax.set_ylabel('Real Power Loss (p.u.)')
ax.set_title('Comparison of Real Power Loss between Non-control and DRL-IMPALA')
ax.set_xticks(x)
ax.set_xticklabels(df['Step'])
ax.legend()

# Display plot
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plotting Reactive Power Loss in a similar style
fig, ax = plt.subplots(figsize=(14, 8))

# Bars for Reactive Power Loss
bars3 = ax.bar(x - width/2, df['Non-control Reactive Power Loss (p.u.)'], width, label='Non-control Reactive Power Loss', color='lightcoral')
bars4 = ax.bar(x + width/2, df['DRL-IMPALA Reactive Power Loss (p.u.)'], width, label='DRL-IMPALA Reactive Power Loss', color='red')

# Adding fancy styling
ax.set_xlabel('Step')
ax.set_ylabel('Reactive Power Loss (p.u.)')
ax.set_title('Comparison of Reactive Power Loss between Non-control and DRL-IMPALA')
ax.set_xticks(x)
ax.set_xticklabels(df['Step'])
ax.legend()

# Display plot
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
